In [2]:
from scipy.stats import ttest_ind, ttest_rel
from scipy.stats import norm, uniform
import numpy as np
from matplotlib import pyplot as plt

## Пример некорректного использования независимого теста Стьюдента.  
Мы возьмем две величины X и Y с одинаковым матожиданием. Но они будут зависимы так, что Y всегда равна -X. Поэтому дисперсия их разницы $\mathrm{D}(X - Y) = \mathrm{D}(2 \cdot X) = 4 \cdot \mathrm{D}(X)$. Но независимый тест Стьюдента будет оценивать ее как $\widehat{\mathrm{D}}(X) + \widehat{\mathrm{D}}(Y) \approx 2 \cdot \mathrm{D}(X)$. Это значит, что он будет думать что эмпирическая оценка $\widehat{\mathrm{E}}(X - Y)$ более точна, чем она есть на самом деле и как следствие будет отвергать нулевую гипотезу чаще, чем нужно.

In [5]:
n_simulations = 10 ** 4
sample_size = 100
significance_level = 0.05
distr = norm()

n_rejections = 0
for i in range(n_simulations):
    sample_one = distr.rvs(sample_size)
    sample_two = -sample_one
    pvalue = ttest_ind(sample_one, sample_two).pvalue
    if pvalue < significance_level:
        n_rejections += 1

print(n_rejections / n_simulations)

0.1643


В этом примере корреляция между двумя выборками отрицательна и поэтому независимый тест Стьюдента недооценивает $\mathrm{D}(X - Y) = \mathrm{D}(X) - 2 \mathrm{Cov}(X, Y) + \mathrm{D}(Y) > \mathrm{D}(X) + \mathrm{D}(Y)$ . Это приводит к лишним ложно-положительным результатам.  
Забавно, что ошибиться вниз более чем в 2 раза таким образом нельзя т. к. $\mathrm{D}(X - Y) \leqslant 2 (\mathrm{D}(X) + \mathrm{D}(Y))$ (упражнение).


## Зависимый тест Стьюдента.
Если корреляция между выборками положительна, независимый тест будет переоценивать $\mathrm{D}(X - Y)$ и как следствие будет слишком редко отвергать нулевую гипотезу (и мы будем упускать на самом деле статзначимые эффекты).

Состряпаем на скорую руку пример подобной ситуации из реальных данных:

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

np.random.seed(0)
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

linear_model = LinearRegression().fit(X_train, y_train)
random_forest = RandomForestRegressor(n_estimators=20).fit(X_train, y_train)

linear_model_predictions = linear_model.predict(X_test)
random_forest_predictions = random_forest.predict(X_test)

linear_model_err = (linear_model_predictions - y_test) ** 2
random_forest_err = (random_forest_predictions - y_test) ** 2

In [61]:
print("Среднее в первой выборке:   {}".format(linear_model_err.mean()))
print("Среднее во второй выборке:  {}".format(random_forest_err.mean()))
print("Корреляция: {}".format(np.corrcoef(linear_model_err, random_forest_err)[0, 1]))

Среднее в первой выборке:   27.195965766883234
Среднее во второй выборке:  17.264955427631573
Корреляция: 0.9418284047865157


In [62]:
print("pvalue независимого теста Стьюдента: {}".format(
    ttest_ind(linear_model_err, random_forest_err).pvalue
))

pvalue независимого теста Стьюдента: 0.24695138090525326


In [63]:
print("pvalue зависимого теста Стьюдента: {}".format(
    ttest_rel(linear_model_err, random_forest_err).pvalue
))

pvalue зависимого теста Стьюдента: 1.72490082175373e-05


Как видно ttest_ind упустил статзначимое отличие